# Model 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import numpy as np
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import TransformerEncoder, TransformerEncoderLayer

## initiation:

In [ ]:
class NumpyFolderDataset(datasets):
    def __init__(self, root_dir, transform=None):
        """
        Dataset to handle .npy files organized by class in subdirectories.

        Args:
        - root_dir (str): Root directory containing class subdirectories.
        - transform (callable, optional): Transform to apply to each sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []  # List to store file paths and their labels

        # Traverse through the directory structure
        for class_idx, class_name in enumerate(sorted(os.listdir(root_dir))):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    if file_name.endswith('.npy'):
                        file_path = os.path.join(class_dir, file_name)
                        self.samples.append((file_path, class_idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        np_array = np.load(file_path).astype(np.float32)  # Load the .npy file

        if self.transform:
            np_array = self.transform(np_array)

        return np_array, label

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(x)),  # Convert NumPy array to PyTorch tensor
    transforms.Lambda(lambda x: x.unsqueeze(0)),   # Add channel dimension for CNN
])

# Path to your dataset
dataset_path = r'F:\TheLis\2_Class'

# Create the dataset
dataset = NumpyFolderDataset(root_dir=dataset_path, transform=transform)

# Split into training and test datasets
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verify DataLoader
for batch_data, batch_labels in train_loader:
    print(f"Batch data shape: {batch_data.shape}, Batch labels: {batch_labels.shape}")
    break
